# Setup

In [ ]:
%%bash
pip install -r requirements.txt

## Code setup

In [ ]:
import os
import pandas as pd

List of runs that can be analyzed, each is a clustering method followed by the number of clusters or other relevant parameters

In [ ]:
cwd = os.getcwd()
results_directory = "../results"
results_directory = os.path.join(cwd, results_directory)
for f in os.listdir(results_directory):
    path = os.path.join(results_directory, f)
    if os.path.isdir(path):
        print(f)

## Choose one of the clustering metod from above here

In [ ]:
name_analysis = 'kmeans10'

In [ ]:
result_path = os.path.join(results_directory, name_analysis)
analysis_path = os.path.join(result_path, "analysis")

# Clusters

In [ ]:
pth = os.path.join(result_path, 'results.csv')
clustering_results = pd.read_csv(pth)
clustering_results.head()

Query for a specific drug pair provide names of both drugs in cell below:

In [ ]:
name1 = 'imipramine'
name2 = 'bupropion'
clustering_results[(clustering_results['name1'] == name1) & (clustering_results['name2'] == name2)]

# Side effects analysis

## Choose a level of side effects

In [ ]:
analysis_level = 'soc'

In [ ]:
pth = os.path.join(analysis_path, 'scores_' + analysis_level + '_term.csv')
sideeffect_results = pd.read_csv(pth)
sideeffect_results

## Clusters numbers

In [ ]:
sideeffect_results["cluster"].drop_duplicates().values

In [ ]:
num_clusters = len(sideeffect_results["cluster"].drop_duplicates().values)

## Choose a specific cluster to inspect

In [ ]:
cluster_no = 0

In [ ]:
sideeffect_results[sideeffect_results['cluster'] == cluster_no]

In [ ]:
alpha = '0.005'

# Significance Analysis

## Choose a level of side effects

In [ ]:
analysis_level = 'soc'

In [ ]:
pth = os.path.join(analysis_path, 'significant_' + analysis_level + '_ranks_' + alpha + '.csv')
statistical_results = pd.read_csv(pth)
statistical_results

Look at the significant results summary

In [ ]:
pth = os.path.join(analysis_path, 'significant_summary.csv')
summary_results = pd.read_csv(pth)
summary_results

Query for specific drugs

In [ ]:
name1 = 'imipramine'
name2 = 'bupropion'
summary_results[(summary_results['name1'] == name1) & (summary_results['name2'] == name2)]

# Target distribution for significant clusters

In [ ]:
from experiment.interactive_analysis import InteractiveAnalyzer

## Choose the level, the number of clusters to see and the number of targets per clusters to show

In [ ]:
analysis_level = 'soc'
cluster_number = num_clusters
targets_per_cluster = 5

This may take some time, it needs to load twosides for further analysis

In [ ]:
analyzer = InteractiveAnalyzer(result_path)

In [ ]:
significant_clusters, important_targets = analyzer.get_important_data(analysis_level, cluster_number, targets_per_cluster)

In [ ]:
significant_clusters

## Choose a specific cluster to inspect from the ones in the table above

In [ ]:
cluster_no = 2

In [ ]:
important_targets[cluster_no].describe()

## Choose a target to visualize

In [ ]:
target = 'Potassium Channel'

In [ ]:
important_targets[cluster_no][target].plot.hist()